<a href="https://colab.research.google.com/github/sandhyaparna/NLP/blob/main/%F0%9F%A4%97_Course.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# light version of 🤗
!pip install datasets transformers[sentencepiece]
!pip install torchvision==0.6.1 
!pip install torch==1.5.1 
!pip install torchaudio==0.5.1

# Development version, which comes with all the required dependencies for pretty much any imaginable use case
# !pip install transformers[dev]

In [2]:
import transformers
from transformers import pipeline

By default, this pipeline selects a particular pretrained model that has been fine-tuned for sentiment analysis in English. The model is downloaded and cached when you create the classifier object. If you rerun the command, the cached model will be used instead and there is no need to download the model again.

There are three main steps involved when you pass some text to a pipeline:
1. The text is preprocessed into a format the model can understand.
2. The preprocessed inputs are passed to the model.
3. The predictions of the model are post-processed, so you can make sense of them.

#  Transformer models

Learn about:
* How to use the pipeline function to solve NLP tasks such as text generation and classification
* About the Transformer architecture
* How to distinguish between encoder, decoder, and encoder-decoder architectures and use cases

Use cases:
* Classifying whole sentences: Getting the sentiment of a review, detecting if an email is spam, determining if a sentence is grammatically correct or whether two sentences are logically related or not
* Classifying each word in a sentence: Identifying the grammatical components of a sentence (noun, verb, adjective), or the named entities (person, location, organization)
* Generating text content: Completing a prompt with auto-generated text, filling in the blanks in a text with masked words
* Extracting an answer from a text: Given a question and a context, extracting the answer to the question based on the information provided in the context
* Generating a new sentence from an input text: Translating a text into another language, summarizing a text

### Sentiment Analysis
Identifies if a record is positive or negative and the score associated with it

In [3]:
sentiment_analysis = pipeline("sentiment-analysis")

In [4]:
sentiment_analysis([
    "I've been waiting for a HuggingFace course my whole life.", 
    "I hate this so much!"
])

[{'label': 'POSITIVE', 'score': 0.9598048329353333},
 {'label': 'NEGATIVE', 'score': 0.9994558095932007}]

In [5]:
sentences = ["I've been waiting for a HuggingFace course my whole life.",
             "I've been waiting for a HuggingFace course my whole life. It is too good to be true",
             "Dont tell me it is true"]

for i in sentences:
  print(sentiment_analysis(i))

[{'label': 'POSITIVE', 'score': 0.9598048329353333}]
[{'label': 'POSITIVE', 'score': 0.9949657320976257}]
[{'label': 'POSITIVE', 'score': 0.8956868648529053}]


### Zero-shot classification
* classify texts that haven’t been labelled
* This pipeline is called zero-shot because you don’t need to fine-tune the model on your data to use it. It can directly return probability scores for any list of labels you want!
* We’ll start by tackling a more challenging task where we need to classify texts that haven’t been labelled. This is a common scenario in real-world projects because annotating text is usually time-consuming and requires domain expertise. For this use case, the zero-shot-classification pipeline is very powerful: it allows you to specify which labels to use for the classification, so you don’t have to rely on the labels of the pretrained model.

In [6]:
zero_shot_classification = pipeline("zero-shot-classification")

In [7]:
zero_shot_classification(
    "This is a course about the Transformers library",
    candidate_labels=["education", "politics", "business", "NLP", "Natural language processing", "text processing"],
)

{'labels': ['education',
  'NLP',
  'business',
  'Natural language processing',
  'text processing',
  'politics'],
 'scores': [0.5944191217422485,
  0.17135168612003326,
  0.07880786061286926,
  0.07346738874912262,
  0.05138999596238136,
  0.030564000830054283],
 'sequence': 'This is a course about the Transformers library'}

In [8]:
zero_shot_classification(
    "This is about the Transformers library",
    candidate_labels=["education", "politics", "business", "NLP", "Natural language processing", "text processing"],
)

{'labels': ['NLP',
  'education',
  'business',
  'Natural language processing',
  'text processing',
  'politics'],
 'scores': [0.33647558093070984,
  0.1643807291984558,
  0.16385307908058167,
  0.13643918931484222,
  0.12724074721336365,
  0.07161065191030502],
 'sequence': 'This is about the Transformers library'}

### Text generation
use a pipeline to generate some text. The main idea here is that you provide a prompt and the model will auto-complete it by generating the remaining text. This is similar to the predictive text feature that is found on many phones. Text generation involves randomness, so it’s normal if you don’t get the same results as shown below.

In [54]:
from transformers import pipeline

You can control how many different sequences are generated with the argument num_return_sequences and the total length of the output text with the argument max_length

In [56]:
# uses default model
text_generation = pipeline("text-generation")

text_generation("In this course, we will teach you how to", num_return_sequences=2, max_length=25)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to connect with your network and to have more success.\n\nWhat follows is'},
 {'generated_text': 'In this course, we will teach you how to create an interactive version of Firefox as a text editor to create your own Firefox'}]

Any models available in https://huggingface.co/models?pipeline_tag=text-generation can be used

In [57]:
# to choose a specific model from the Hub, use model argument
generator = pipeline("text-generation", model="distilgpt2")
generator(
    "In this course, we will teach you how to",
    max_length=30,
    num_return_sequences=2,
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to identify your feelings and you will learn more about the benefits of taking the easy way out.'},
 {'generated_text': 'In this course, we will teach you how to build a new game-specific system, and the best way to use it. To create new content'}]

### Mask filling
* The idea of this task is to fill in the blanks in a given text
* top_k argument controls how many possibilities you want to be displayed. Note that here the model fills in the special <mask> word, which is often referred to as a mask token. Other mask-filling models might have different mask tokens, so it’s always good to verify the proper mask word when exploring other models. 

In [59]:
fill_mask_unmasker = pipeline("fill-mask")
fill_mask_unmasker("This course will teach you all about <mask> models.", top_k=2)

[{'score': 0.19619838893413544,
  'sequence': 'This course will teach you all about mathematical models.',
  'token': 30412,
  'token_str': ' mathematical'},
 {'score': 0.040527306497097015,
  'sequence': 'This course will teach you all about computational models.',
  'token': 38163,
  'token_str': ' computational'}]

In [ ]:
bert_base_unmasker = pipeline('fill-mask', model='bert-base-cased')
bert_base_unmasker("Hello I'm a [MASK] model.")

In [53]:
# Universal sentence embedding of Google implemented in Transformers
# Understand how tokenization is different when sentences are seperated by space versus fullstop

# https://huggingface.co/johngiorgi/declutr-small
import torch
from scipy.spatial.distance import cosine

from transformers import AutoModel, AutoTokenizer

# Load the model
tokenizer = AutoTokenizer.from_pretrained("johngiorgi/declutr-small")
model = AutoModel.from_pretrained("johngiorgi/declutr-small")

# Prepare some text to embed
text = [
    "A smiling costumed woman is holding an umbrella. A happy woman in a fairy costume holds an umbrella.",
    "A smiling costumed woman is holding an umbrella A happy woman in a fairy costume holds an umbrella.",
]
inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
print("\ninputs:",inputs)

# detokenize - to understand tokenization
# print(tokenizer.decode(inputs["input_ids"][0]))
# print(tokenizer.decode(inputs["input_ids"][0]))
for i in range(len(inputs["input_ids"])):
  print("\ndetokenized:",tokenizer.decode(inputs["input_ids"][i]))

# Embed the text
with torch.no_grad():
    sequence_output = model(**inputs)[0]

# Mean pool the token-level embeddings to get sentence-level embeddings
embeddings = torch.sum(
    sequence_output * inputs["attention_mask"].unsqueeze(-1), dim=1
) / torch.clamp(torch.sum(inputs["attention_mask"], dim=1, keepdims=True), min=1e-9)

# Compute a semantic similarity via the cosine distance
semantic_sim = 1 - cosine(embeddings[0], embeddings[1])
print("\nsemantic similarity:",semantic_sim)

Some weights of the model checkpoint at johngiorgi/declutr-small were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



inputs: {'input_ids': tensor([[    0,   250, 12382,   701, 28817,   693,    16,  1826,    41, 16676,
             4,    83,  1372,   693,    11,    10, 25310, 12111,  3106,    41,
         16676,     4,     2],
        [    0,   250, 12382,   701, 28817,   693,    16,  1826,    41, 16676,
            83,  1372,   693,    11,    10, 25310, 12111,  3106,    41, 16676,
             4,     2,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])}

detokenized: <s>A smiling costumed woman is holding an umbrella. A happy woman in a fairy costume holds an umbrella.</s>

detokenized: <s>A smiling costumed woman is holding an umbrella A happy woman in a fairy costume holds an umbrella.</s><pad>

semantic similarity: 0.9911488890647888
